In [ ]:
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import pprint
from __future__ import division
from __future__ import print_function
import calendar
from zipfile import ZipFile
import os
import sys
import dateutil
import zipfile
import csv
import pickle
import datetime

## Define search functions 

In [ ]:
def collect_routerData(routerDataFile , base_data=pd.DataFrame()):
    filePath = routerDataFile
    directionIsIn = False
    data = base_data.copy(deep=True)
    group = ""
    routerNum = 0
    with open(filePath, "rb") as csvfile:
        csvReader = csv.reader(csvfile, delimiter=',')
        for row in csvReader:
            #print(group_and_router + str(directionIsIn))
            # check if the line defines our router
            if len(row) == 1 and "cern.ch" in row[0]:
                routerNum += 1
                continue
            # check if there is actually data
            if len(row) != 2:
                continue
            # check if the line defines our user group
            if row[0] == "Subtitle:":
                group = row[1].split("User Group: ")[1]
            # check the direction
            if row[1] == "Bits In - Average Rate (Bits Per Second)":
                directionIsIn = True
            if row[1] == "Bits Out - Average Rate (Bits Per Second)":
                directionIsIn = False
            try:
                float(row[1])
            except ValueError:
                continue
            bitsPerSec = float(row[1])
            row[0] = row[0].replace(" ", "/")
            # the format for the years is changed inbetween from 16 to 2016
            year = row[0].split("/")[2]
            if len(year) == 4:
                timestomp = datetime.strptime(row[0], "%m/%d/%Y/%H:%M")
            else:
                timestomp = datetime.strptime(row[0], "%m/%d/%y/%H:%M")
            print(timestomp)
            # add data to dataFrame
            timestamp_epoch = int(calendar.timegm(timestomp.utctimetuple()))
            if directionIsIn:
                data.set_value(timestamp_epoch, "throughput_"+group+"_router_"+str(routerNum)+"_in", bitsPerSec)
            else:
                data.set_value(timestamp_epoch, "throughput_"+group+"_router_"+str(routerNum)+"_out", bitsPerSec)
    data.sort_index(inplace=True, ascending=False)
    return data


In [ ]:
def starlikeConnector(es, center_point_data, out_points_list, timeframe):
    print("starlikeConnector not implementet for this!")
    return
    for dest_data in out_points_list:
        collect_and_clean_byIP_Scroll(es, center_point_data, dest_data, timeframe)
        collect_and_clean_byIP_Scroll(es, dest_data, center_point_data, timeframe)

        
def zipFolder(path="raw_data/"):
    zipf = zipfile.ZipFile('folder_zipped.zip', 'w', zipfile.ZIP_DEFLATED)
    for root, dirs, files in os.walk(path):
        for file in files:
            zipf.write(os.path.join(root, file))
    zipf.close()
        
def readAllInFolder(folderPath, base_data=pd.DataFrame()):
    data = base_data
    for (dirpath, dirnames, filenames) in os.walk(folderPath):
        # read all files
        for filename in filenames:
            data = collect_routerData(folderPath + filename, base_data=data)
    return data

## Collect raw data

In [ ]:
data_router = readAllInFolder("raw_data_routers/PIC/")
data_router = readAllInFolder("raw_data_routers/RAL/", base_data=data_router)
data_router.to_pickle("raw_data_routers/raw_router_RAL_and_PIC_all_highres.pkl")

## plotting

In [ ]:
src = "CERN"
dest = "PIC"
# plot the data we got
plt.figure(figsize=(12,9), dpi=300)
title = "Throughput plot "+src+" <-> "+dest
fname = title
plt.title(title,fontsize=14)
# plot latency 1
plt.plot(np.array(data_router.axes[0]), data_router["throughput_UK-RAL_router_1_in"],
         'r', label=r'Throughput, router 1: '+dest+' -> '+src)
#plt.plot(np.array(latency_data.axes[0]), latency_data["delay_forth_minus_avg"], 'r', label=r'Latency '+src+' -> '+dest)
# plot latency 2
plt.plot(np.array(data_router.axes[0]), data_router["throughput_UK-RAL_router_1_out"],
         'b', label=r'Throughput, router 1: '+src+' -> '+dest)
#plt.plot(np.array(data_router.axes[0]), latency_data["delay_back_minus_avg"], 'b', label=r'Latency '+dest+' -> '+src)
# test
#plt.plot(np.array(data_router.axes[0]), data_router["throughput_ES-PIC_router_1_out"], 'g', label=r'Test')
plt.legend(loc='best',fontsize=14)
plt.ylabel("Throughput")
plt.xlabel("Time")
#plt.axis([1.4712*1e9, 1.4714*1e9, -0, 80])
#plt.axis([1.4634*1e9, 1.4641*1e9, 0, 100])
#plt.savefig('own_plots/'+ fname +'.png', format='PNG', dpi=300)
plt.show()